In [1]:
import pandas as pd
import pydicom
from matplotlib import pyplot as plt
import png

## Обрабатываем снимки DICOM СС в TEST

### Просмотр описания снимков

In [104]:
### Просмотрим данные по снимкам в Calc-Test
data_test=pd.read_csv('test_xran.csv')

## Отбираем только с видом CC
data_test=data_test.loc[data_test['image view']=='MLO'].reset_index().drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'index'])
data_test.head()

,cropped image file path,abnormality id,abnormality type,assessment,breast density,breast_density,calc distribution,calc type,ROI mask file path,image file path,image view,left or right breast,mass margins,mass shape,pathology,patient_id,subtlety
0,F:/CBIS-DDSM/Calc-Test_P_00038_LEFT_MLO_1\08-2...,1,calcification,4,2.0,NaN,CLUSTERED,PUNCTATE-PLEOMORPHIC,F:/CBIS-DDSM/Calc-Test_P_00038_LEFT_MLO_1\08-2...,F:/CBIS-DDSM/Calc-Test_P_00038_LEFT_MLO\08-29-...,MLO,LEFT,NaN,NaN,BENIGN,P_00038,2
1,F:/CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO_1\08-...,1,calcification,2,2.0,NaN,NaN,VASCULAR,F:/CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO_1\08-...,F:/CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO\08-29...,MLO,RIGHT,NaN,NaN,BENIGN_WITHOUT_CALLBACK,P_00038,5
2,F:/CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO_2\08-...,2,calcification,2,2.0,NaN,NaN,VASCULAR,F:/CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO_2\08-...,F:/CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO\08-29...,MLO,RIGHT,NaN,NaN,BENIGN_WITHOUT_CALLBACK,P_00038,5
3,F:/CBIS-DDSM/Calc-Test_P_00041_LEFT_MLO_2\08-2...,2,calcification,2,1.0,NaN,NaN,LUCENT_CENTER,F:/CBIS-DDSM/Calc-Test_P_00041_LEFT_MLO_2\08-2...,F:/CBIS-DDSM/Calc-Test_P_00041_LEFT_MLO\08-29-...,MLO,LEFT,NaN,NaN,BENIGN_WITHOUT_CALLBACK,P_00041,5
4,F:/CBIS-DDSM/Calc-Test_P_00077_LEFT_MLO_1\08-2...,1,calcification,2,2.0,NaN,NaN,ROUND_AND_REGULAR,F:/CBIS-DDSM/Calc-Test_P_00077_LEFT_MLO_1\08-2...,F:/CBIS-DDSM/Calc-Test_P_00077_LEFT_MLO\08-29-...,MLO,LEFT,NaN,NaN,BENIGN_WITHOUT_CALLBACK,P_00077,3


In [105]:
len(data_test)

378

In [83]:
len(data_test[data_test['assessment']==5])

250

### Конвертируем (DCM2PNG), отзеркаливаем снимки RIGHT и сохраняем снимки в нужных папках с классами 

In [84]:
import pydicom
import matplotlib.pyplot as plt
import scipy.misc
from skimage import io, transform, feature
from PIL import Image
import os
import re
import numpy as np

In [91]:
## Функция №1 аугментации для PNG:
def func_aug1(path_file_save):
    im = Image.open(path_file_save)
    im = im.transpose(Image.FLIP_LEFT_RIGHT)
    return im

In [92]:
## Функция №2 аугментации для PNG:
def func_aug2(path_file_save):
    im = Image.open(path_file_save)
    im = im.rotate(5)    
    return im

In [93]:
## Функция №3 аугментации для PNG:
def func_aug3(path_file_save):
    im = Image.open(path_file_save)
    im = im.transpose(Image.FLIP_LEFT_RIGHT)
    im = im.rotate(5)     
    return im

In [94]:
## Функция №4 аугментации для PNG:
def func_aug4(path_file_save):
    im = Image.open(path_file_save)
    im = im.rotate(-5) 
    return im

In [95]:
## Функция №5 аугментации для PNG:
def func_aug5(path_file_save):
    im = Image.open(path_file_save)
    im = im.transpose(Image.FLIP_LEFT_RIGHT)
    im = im.rotate(-5) 
    return im

In [96]:
## Функция №6 аугментации для PNG:
def func_aug6(path_file_save):
    im = Image.open(path_file_save)
    im = im.rotate(10)     
    return im

In [97]:
## Функция №7 аугментации для PNG:
def func_aug7(path_file_save):
    im = Image.open(path_file_save)
    im = im.transpose(Image.FLIP_LEFT_RIGHT)
    im = im.rotate(10) 
    return im

In [108]:
## Функция для конвертации снимка из DICOM в PNG
def convert_to_png(file, category, number):
    ds = pydicom.dcmread(file)

    shape = ds.pixel_array.shape

    # Convert to float to avoid overflow or underflow losses.
    image_2d = ds.pixel_array.astype(float)

    # Rescaling grey scale between 0-255
    image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0

    # Convert to uint
    image_2d_scaled = np.uint8(image_2d_scaled)

    # Write the PNG file
    path_file_save = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('.png')
    with open(path_file_save, 'wb') as png_file:
        w = png.Writer(shape[1], shape[0], greyscale=True)
        w.write(png_file, image_2d_scaled)
    if category == '5':
        # Делаем аугментацию №1 и 2 и сохраняем файлы
        img = func_aug1(path_file_save)
        path_file_save_aug1 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug1.png')
        img.save(path_file_save_aug1)
        img = func_aug2(path_file_save)
        path_file_save_aug2 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug2.png')
        img.save(path_file_save_aug2) 
    elif category == '2':
        # Делаем аугментацию №1, 2 и 3 и сохраняем файлы
        img = func_aug1(path_file_save)
        path_file_save_aug1 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug1.png')
        img.save(path_file_save_aug1)
        img = func_aug2(path_file_save)
        path_file_save_aug2 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug2.png')
        img.save(path_file_save_aug2)
        img = func_aug3(path_file_save)
        path_file_save_aug3 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug3.png')
        img.save(path_file_save_aug3)
    elif category == '3':
        # Делаем аугментацию №1, 2 и 3 и сохраняем файлы
        img = func_aug1(path_file_save)
        path_file_save_aug1 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug1.png')
        img.save(path_file_save_aug1)
        img = func_aug2(path_file_save)
        path_file_save_aug2 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug2.png')
        img.save(path_file_save_aug2)
        img = func_aug3(path_file_save)
        path_file_save_aug3 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug3.png')
        img.save(path_file_save_aug3)
    elif category == '0':
        # Делаем аугментацию №1-7 и сохраняем файлы
        img = func_aug1(path_file_save)
        path_file_save_aug1 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug1.png')
        img.save(path_file_save_aug1)
        
        img = func_aug2(path_file_save)
        path_file_save_aug2 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug2.png')
        img.save(path_file_save_aug2)
        
        img = func_aug3(path_file_save)
        path_file_save_aug3 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug3.png')
        img.save(path_file_save_aug3)
        
        img = func_aug4(path_file_save)
        path_file_save_aug4 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug4.png')
        img.save(path_file_save_aug4)
        
        img = func_aug5(path_file_save)
        path_file_save_aug5 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug5.png')
        img.save(path_file_save_aug5)
        
        img = func_aug6(path_file_save)
        path_file_save_aug6 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug6.png')
        img.save(path_file_save_aug6)
        
        img = func_aug7(path_file_save)
        path_file_save_aug7 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug7.png')
        img.save(path_file_save_aug7)
    else:
        pass

In [110]:
## Функция для конвертации снимка из DICOM в PNG
def convert_to_png_mlo(file, category, number):
    ds = pydicom.dcmread(file)

    shape = ds.pixel_array.shape

    # Convert to float to avoid overflow or underflow losses.
    image_2d = ds.pixel_array.astype(float)

    # Rescaling grey scale between 0-255
    image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0

    # Convert to uint
    image_2d_scaled = np.uint8(image_2d_scaled)

    # Write the PNG file
    path_file_save = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('.png')
    with open(path_file_save, 'wb') as png_file:
        w = png.Writer(shape[1], shape[0], greyscale=True)
        w.write(png_file, image_2d_scaled)
    if category == '5':
        # Делаем аугментацию №1, 2 и 3 и сохраняем файлы
        img = func_aug1(path_file_save)
        path_file_save_aug1 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug1.png')
        img.save(path_file_save_aug1)
        img = func_aug2(path_file_save)
        path_file_save_aug2 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug2.png')
        img.save(path_file_save_aug2)
        img = func_aug3(path_file_save)
        path_file_save_aug3 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug3.png')
        img.save(path_file_save_aug3)
    elif category == '2':
        # Делаем аугментацию №1, 2 и 3 и сохраняем файлы
        img = func_aug1(path_file_save)
        path_file_save_aug1 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug1.png')
        img.save(path_file_save_aug1)
        img = func_aug2(path_file_save)
        path_file_save_aug2 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug2.png')
        img.save(path_file_save_aug2)
        img = func_aug3(path_file_save)
        path_file_save_aug3 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug3.png')
        img.save(path_file_save_aug3)
    elif category == '3':
        # Делаем аугментацию №1, 2 и 3 и сохраняем файлы
        img = func_aug1(path_file_save)
        path_file_save_aug1 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug1.png')
        img.save(path_file_save_aug1)
        img = func_aug2(path_file_save)
        path_file_save_aug2 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug2.png')
        img.save(path_file_save_aug2)
        img = func_aug3(path_file_save)
        path_file_save_aug3 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug3.png')
        img.save(path_file_save_aug3)
    elif category == '0':
        # Делаем аугментацию №1-7 и сохраняем файлы
        img = func_aug1(path_file_save)
        path_file_save_aug1 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug1.png')
        img.save(path_file_save_aug1)
        
        img = func_aug2(path_file_save)
        path_file_save_aug2 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug2.png')
        img.save(path_file_save_aug2)
        
        img = func_aug3(path_file_save)
        path_file_save_aug3 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug3.png')
        img.save(path_file_save_aug3)
        
        img = func_aug4(path_file_save)
        path_file_save_aug4 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug4.png')
        img.save(path_file_save_aug4)
        
        img = func_aug5(path_file_save)
        path_file_save_aug5 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug5.png')
        img.save(path_file_save_aug5)
        
        img = func_aug6(path_file_save)
        path_file_save_aug6 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug6.png')
        img.save(path_file_save_aug6)
        
        img = func_aug7(path_file_save)
        path_file_save_aug7 = 'F:\\dataset4\\test\\'+str(category)+'\\'+str(number)+str('_aug7.png')
        img.save(path_file_save_aug7)
    else:
        pass

In [111]:
for i in range(len(data_test)):
    path=data_test['image file path'][i]
    file = path.replace('/', '\\\\')
    category = str(data_test['assessment'][i])
    number = 'calc_'+str(i)
    convert_to_png_mlo(file, category, number)
    if i%5==0:
        print('Сохранено {} снимков из {}'.format(str(i), str(len(data_test))))

Сохранено 0 снимков из 378
Сохранено 5 снимков из 378
Сохранено 10 снимков из 378
Сохранено 15 снимков из 378
Сохранено 20 снимков из 378
Сохранено 25 снимков из 378
Сохранено 30 снимков из 378
Сохранено 35 снимков из 378
Сохранено 40 снимков из 378
Сохранено 45 снимков из 378
Сохранено 50 снимков из 378
Сохранено 55 снимков из 378
Сохранено 60 снимков из 378
Сохранено 65 снимков из 378
Сохранено 70 снимков из 378
Сохранено 75 снимков из 378
Сохранено 80 снимков из 378
Сохранено 85 снимков из 378
Сохранено 90 снимков из 378
Сохранено 95 снимков из 378
Сохранено 100 снимков из 378
Сохранено 105 снимков из 378
Сохранено 110 снимков из 378
Сохранено 115 снимков из 378
Сохранено 120 снимков из 378
Сохранено 125 снимков из 378
Сохранено 130 снимков из 378
Сохранено 135 снимков из 378
Сохранено 140 снимков из 378
Сохранено 145 снимков из 378
Сохранено 150 снимков из 378
Сохранено 155 снимков из 378
Сохранено 160 снимков из 378
Сохранено 165 снимков из 378
Сохранено 170 снимков из 378
Сохране

In [112]:
### Просмотрим данные по снимкам в Train
data_train=pd.read_csv('train_xran.csv')

## Отбираем только с видом CC
data_train=data_train.loc[data_train['image view']=='CC'].reset_index().drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'index'])
data_train.head()

,ROI mask file path,abnormality id,abnormality type,assessment,breast density,breast_density,calc distribution,calc type,cropped image file path,image file path,image view,left or right breast,mass margins,mass shape,pathology,patient_id,subtlety
0,F:/CBIS-DDSM/Calc-Training_P_00005_RIGHT_CC_1\...,1,calcification,3,3.0,NaN,CLUSTERED,AMORPHOUS,F:\CBIS-DDSM\Calc-Training_P_00005_RIGHT_CC_1\...,F:/CBIS-DDSM/Calc-Training_P_00005_RIGHT_CC\08...,CC,RIGHT,NaN,NaN,MALIGNANT,P_00005,3
1,F:/CBIS-DDSM/Calc-Training_P_00007_LEFT_CC_1\0...,1,calcification,4,4.0,NaN,LINEAR,PLEOMORPHIC,F:\CBIS-DDSM\Calc-Training_P_00007_LEFT_CC_1\0...,F:/CBIS-DDSM/Calc-Training_P_00007_LEFT_CC\08-...,CC,LEFT,NaN,NaN,BENIGN,P_00007,4
2,F:/CBIS-DDSM/Calc-Training_P_00008_LEFT_CC_1\0...,1,calcification,2,1.0,NaN,REGIONAL,NaN,F:\CBIS-DDSM\Calc-Training_P_00008_LEFT_CC_1\0...,F:/CBIS-DDSM/Calc-Training_P_00008_LEFT_CC\08-...,CC,LEFT,NaN,NaN,BENIGN_WITHOUT_CALLBACK,P_00008,3
3,F:/CBIS-DDSM/Calc-Training_P_00008_LEFT_CC_2\0...,2,calcification,2,1.0,NaN,REGIONAL,NaN,F:\CBIS-DDSM\Calc-Training_P_00008_LEFT_CC_2\0...,F:/CBIS-DDSM/Calc-Training_P_00008_LEFT_CC\08-...,CC,LEFT,NaN,NaN,BENIGN_WITHOUT_CALLBACK,P_00008,3
4,F:/CBIS-DDSM/Calc-Training_P_00008_LEFT_CC_3\0...,3,calcification,2,1.0,NaN,REGIONAL,NaN,F:\CBIS-DDSM\Calc-Training_P_00008_LEFT_CC_3\0...,F:/CBIS-DDSM/Calc-Training_P_00008_LEFT_CC\08-...,CC,LEFT,NaN,NaN,BENIGN_WITHOUT_CALLBACK,P_00008,3


In [113]:
## Функция для конвертации снимка из DICOM в PNG
def convert_to_png_cc_train(file, category, number):
    ds = pydicom.dcmread(file)

    shape = ds.pixel_array.shape

    # Convert to float to avoid overflow or underflow losses.
    image_2d = ds.pixel_array.astype(float)

    # Rescaling grey scale between 0-255
    image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0

    # Convert to uint
    image_2d_scaled = np.uint8(image_2d_scaled)

    # Write the PNG file
    path_file_save = 'F:\\dataset3\\train\\'+str(category)+'\\'+str(number)+str('.png')
    with open(path_file_save, 'wb') as png_file:
        w = png.Writer(shape[1], shape[0], greyscale=True)
        w.write(png_file, image_2d_scaled)
    if category == '5':
        # Делаем аугментацию №1, 2 и 3 и сохраняем файлы
        img = func_aug1(path_file_save)
        path_file_save_aug1 = 'F:\\dataset3\\train\\'+str(category)+'\\'+str(number)+str('_aug1.png')
        img.save(path_file_save_aug1)
        img = func_aug2(path_file_save)
        path_file_save_aug2 = 'F:\\dataset3\\train\\'+str(category)+'\\'+str(number)+str('_aug2.png')
        img.save(path_file_save_aug2)
    elif category == '2':
        # Делаем аугментацию №1, 2 и 3 и сохраняем файлы
        img = func_aug1(path_file_save)
        path_file_save_aug1 = 'F:\\dataset3\\train\\'+str(category)+'\\'+str(number)+str('_aug1.png')
        img.save(path_file_save_aug1)
    elif category == '3':
        # Делаем аугментацию №1, 2 и 3 и сохраняем файлы
        img = func_aug1(path_file_save)
        path_file_save_aug1 = 'F:\\dataset3\\train\\'+str(category)+'\\'+str(number)+str('_aug1.png')
        img.save(path_file_save_aug1)
        img = func_aug2(path_file_save)
        path_file_save_aug2 = 'F:\\dataset3\\train\\'+str(category)+'\\'+str(number)+str('_aug2.png')
        img.save(path_file_save_aug2)
        img = func_aug3(path_file_save)
        path_file_save_aug3 = 'F:\\dataset3\\train\\'+str(category)+'\\'+str(number)+str('_aug3.png')
        img.save(path_file_save_aug3)
    elif category == '0':
        # Делаем аугментацию №1-7 и сохраняем файлы
        img = func_aug1(path_file_save)
        path_file_save_aug1 = 'F:\\dataset3\\train\\'+str(category)+'\\'+str(number)+str('_aug1.png')
        img.save(path_file_save_aug1)
        
        img = func_aug2(path_file_save)
        path_file_save_aug2 = 'F:\\dataset3\\train\\'+str(category)+'\\'+str(number)+str('_aug2.png')
        img.save(path_file_save_aug2)
        
        img = func_aug3(path_file_save)
        path_file_save_aug3 = 'F:\\dataset3\\train\\'+str(category)+'\\'+str(number)+str('_aug3.png')
        img.save(path_file_save_aug3)
        
        img = func_aug4(path_file_save)
        path_file_save_aug4 = 'F:\\dataset3\\train\\'+str(category)+'\\'+str(number)+str('_aug4.png')
        img.save(path_file_save_aug4)
        
        img = func_aug5(path_file_save)
        path_file_save_aug5 = 'F:\\dataset3\\train\\'+str(category)+'\\'+str(number)+str('_aug5.png')
        img.save(path_file_save_aug5)
    else:
        pass

In [114]:
for i in range(len(data_train)):
    path=data_train['image file path'][i]
    file = path.replace('/', '\\\\')
    category = str(data_train['assessment'][i])
    number = 'calc_'+str(i)
    convert_to_png_cc_train(file, category, number)
    if i%5==0:
        print('Сохранено {} снимков из {}'.format(str(i), str(len(data_train))))

Сохранено 0 снимков из 1346
Сохранено 5 снимков из 1346
Сохранено 10 снимков из 1346
Сохранено 15 снимков из 1346
Сохранено 20 снимков из 1346
Сохранено 25 снимков из 1346
Сохранено 30 снимков из 1346
Сохранено 35 снимков из 1346
Сохранено 40 снимков из 1346
Сохранено 45 снимков из 1346
Сохранено 50 снимков из 1346
Сохранено 55 снимков из 1346
Сохранено 60 снимков из 1346
Сохранено 65 снимков из 1346
Сохранено 70 снимков из 1346
Сохранено 75 снимков из 1346
Сохранено 80 снимков из 1346
Сохранено 85 снимков из 1346
Сохранено 90 снимков из 1346
Сохранено 95 снимков из 1346
Сохранено 100 снимков из 1346
Сохранено 105 снимков из 1346
Сохранено 110 снимков из 1346
Сохранено 115 снимков из 1346
Сохранено 120 снимков из 1346
Сохранено 125 снимков из 1346
Сохранено 130 снимков из 1346
Сохранено 135 снимков из 1346
Сохранено 140 снимков из 1346
Сохранено 145 снимков из 1346
Сохранено 150 снимков из 1346
Сохранено 155 снимков из 1346
Сохранено 160 снимков из 1346
Сохранено 165 снимков из 1346
Со

In [115]:
### Просмотрим данные по снимкам в Train
data_train_mlo=pd.read_csv('train_xran.csv')

## Отбираем только с видом CC
data_train_mlo=data_train_mlo.loc[data_train_mlo['image view']=='MLO'].reset_index().drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'index'])
data_train_mlo.head()

,ROI mask file path,abnormality id,abnormality type,assessment,breast density,breast_density,calc distribution,calc type,cropped image file path,image file path,image view,left or right breast,mass margins,mass shape,pathology,patient_id,subtlety
0,F:/CBIS-DDSM/Calc-Training_P_00005_RIGHT_MLO_1...,1,calcification,3,3.0,NaN,CLUSTERED,AMORPHOUS,F:\CBIS-DDSM\Calc-Training_P_00005_RIGHT_MLO_1...,F:/CBIS-DDSM/Calc-Training_P_00005_RIGHT_MLO\0...,MLO,RIGHT,NaN,NaN,MALIGNANT,P_00005,3
1,F:/CBIS-DDSM/Calc-Training_P_00007_LEFT_MLO_1\...,1,calcification,4,4.0,NaN,LINEAR,PLEOMORPHIC,F:\CBIS-DDSM\Calc-Training_P_00007_LEFT_MLO_1\...,F:/CBIS-DDSM/Calc-Training_P_00007_LEFT_MLO\08...,MLO,LEFT,NaN,NaN,BENIGN,P_00007,4
2,F:/CBIS-DDSM/Calc-Training_P_00008_LEFT_MLO_1\...,1,calcification,2,1.0,NaN,REGIONAL,NaN,F:\CBIS-DDSM\Calc-Training_P_00008_LEFT_MLO_1\...,F:/CBIS-DDSM/Calc-Training_P_00008_LEFT_MLO\08...,MLO,LEFT,NaN,NaN,BENIGN_WITHOUT_CALLBACK,P_00008,3
3,F:/CBIS-DDSM/Calc-Training_P_00008_LEFT_MLO_2\...,2,calcification,2,1.0,NaN,REGIONAL,NaN,F:\CBIS-DDSM\Calc-Training_P_00008_LEFT_MLO_2\...,F:/CBIS-DDSM/Calc-Training_P_00008_LEFT_MLO\08...,MLO,LEFT,NaN,NaN,BENIGN_WITHOUT_CALLBACK,P_00008,3
4,F:/CBIS-DDSM/Calc-Training_P_00008_LEFT_MLO_3\...,3,calcification,2,1.0,NaN,REGIONAL,NaN,F:\CBIS-DDSM\Calc-Training_P_00008_LEFT_MLO_3\...,F:/CBIS-DDSM/Calc-Training_P_00008_LEFT_MLO\08...,MLO,LEFT,NaN,NaN,BENIGN_WITHOUT_CALLBACK,P_00008,3


In [116]:
## Функция для конвертации снимка из DICOM в PNG
def convert_to_png_mlo_train(file, category, number):
    ds = pydicom.dcmread(file)

    shape = ds.pixel_array.shape

    # Convert to float to avoid overflow or underflow losses.
    image_2d = ds.pixel_array.astype(float)

    # Rescaling grey scale between 0-255
    image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0

    # Convert to uint
    image_2d_scaled = np.uint8(image_2d_scaled)

    # Write the PNG file
    path_file_save = 'F:\\dataset4\\train\\'+str(category)+'\\'+str(number)+str('.png')
    with open(path_file_save, 'wb') as png_file:
        w = png.Writer(shape[1], shape[0], greyscale=True)
        w.write(png_file, image_2d_scaled)
    if category == '5':
        # Делаем аугментацию №1, 2 и 3 и сохраняем файлы
        img = func_aug1(path_file_save)
        path_file_save_aug1 = 'F:\\dataset4\\train\\'+str(category)+'\\'+str(number)+str('_aug1.png')
        img.save(path_file_save_aug1)
        img = func_aug2(path_file_save)
        path_file_save_aug2 = 'F:\\dataset4\\train\\'+str(category)+'\\'+str(number)+str('_aug2.png')
        img.save(path_file_save_aug2)
    elif category == '2':
        # Делаем аугментацию №1, 2 и 3 и сохраняем файлы
        img = func_aug1(path_file_save)
        path_file_save_aug1 = 'F:\\dataset4\\train\\'+str(category)+'\\'+str(number)+str('_aug1.png')
        img.save(path_file_save_aug1)
        img = func_aug2(path_file_save)
        path_file_save_aug2 = 'F:\\dataset4\\train\\'+str(category)+'\\'+str(number)+str('_aug2.png')
        img.save(path_file_save_aug2)
    elif category == '3':
        # Делаем аугментацию №1, 2 и 3 и сохраняем файлы
        img = func_aug1(path_file_save)
        path_file_save_aug1 = 'F:\\dataset4\\train\\'+str(category)+'\\'+str(number)+str('_aug1.png')
        img.save(path_file_save_aug1)
        img = func_aug2(path_file_save)
        path_file_save_aug2 = 'F:\\dataset4\\train\\'+str(category)+'\\'+str(number)+str('_aug2.png')
        img.save(path_file_save_aug2)
        img = func_aug3(path_file_save)
        path_file_save_aug3 = 'F:\\dataset4\\train\\'+str(category)+'\\'+str(number)+str('_aug3.png')
        img.save(path_file_save_aug3)
    elif category == '0':
        # Делаем аугментацию №1-7 и сохраняем файлы
        img = func_aug1(path_file_save)
        path_file_save_aug1 = 'F:\\dataset4\\train\\'+str(category)+'\\'+str(number)+str('_aug1.png')
        img.save(path_file_save_aug1)
        
        img = func_aug2(path_file_save)
        path_file_save_aug2 = 'F:\\dataset4\\train\\'+str(category)+'\\'+str(number)+str('_aug2.png')
        img.save(path_file_save_aug2)
        
        img = func_aug3(path_file_save)
        path_file_save_aug3 = 'F:\\dataset4\\train\\'+str(category)+'\\'+str(number)+str('_aug3.png')
        img.save(path_file_save_aug3)
        
        img = func_aug4(path_file_save)
        path_file_save_aug4 = 'F:\\dataset4\\train\\'+str(category)+'\\'+str(number)+str('_aug4.png')
        img.save(path_file_save_aug4)
        
        img = func_aug5(path_file_save)
        path_file_save_aug5 = 'F:\\dataset4\\train\\'+str(category)+'\\'+str(number)+str('_aug5.png')
        img.save(path_file_save_aug5)
        
        img = func_aug6(path_file_save)
        path_file_save_aug6 = 'F:\\dataset4\\train\\'+str(category)+'\\'+str(number)+str('_aug6.png')
        img.save(path_file_save_aug6)
    else:
        pass

In [117]:
for i in range(len(data_train_mlo)):
    path=data_train_mlo['image file path'][i]
    file = path.replace('/', '\\\\')
    category = str(data_train_mlo['assessment'][i])
    number = 'calc_'+str(i)
    convert_to_png_mlo_train(file, category, number)
    if i%5==0:
        print('Сохранено {} снимков из {}'.format(str(i), str(len(data_train_mlo))))

Сохранено 0 снимков из 1518
Сохранено 5 снимков из 1518
Сохранено 10 снимков из 1518
Сохранено 15 снимков из 1518
Сохранено 20 снимков из 1518
Сохранено 25 снимков из 1518
Сохранено 30 снимков из 1518
Сохранено 35 снимков из 1518
Сохранено 40 снимков из 1518
Сохранено 45 снимков из 1518
Сохранено 50 снимков из 1518
Сохранено 55 снимков из 1518
Сохранено 60 снимков из 1518
Сохранено 65 снимков из 1518
Сохранено 70 снимков из 1518
Сохранено 75 снимков из 1518
Сохранено 80 снимков из 1518
Сохранено 85 снимков из 1518
Сохранено 90 снимков из 1518
Сохранено 95 снимков из 1518
Сохранено 100 снимков из 1518
Сохранено 105 снимков из 1518
Сохранено 110 снимков из 1518
Сохранено 115 снимков из 1518
Сохранено 120 снимков из 1518
Сохранено 125 снимков из 1518
Сохранено 130 снимков из 1518
Сохранено 135 снимков из 1518
Сохранено 140 снимков из 1518
Сохранено 145 снимков из 1518
Сохранено 150 снимков из 1518
Сохранено 155 снимков из 1518
Сохранено 160 снимков из 1518
Сохранено 165 снимков из 1518
Со

Сохранено 1360 снимков из 1518
Сохранено 1365 снимков из 1518
Сохранено 1370 снимков из 1518
Сохранено 1375 снимков из 1518
Сохранено 1380 снимков из 1518
Сохранено 1385 снимков из 1518
Сохранено 1390 снимков из 1518
Сохранено 1395 снимков из 1518
Сохранено 1400 снимков из 1518
Сохранено 1405 снимков из 1518
Сохранено 1410 снимков из 1518
Сохранено 1415 снимков из 1518
Сохранено 1420 снимков из 1518
Сохранено 1425 снимков из 1518
Сохранено 1430 снимков из 1518
Сохранено 1435 снимков из 1518
Сохранено 1440 снимков из 1518
Сохранено 1445 снимков из 1518
Сохранено 1450 снимков из 1518
Сохранено 1455 снимков из 1518
Сохранено 1460 снимков из 1518
Сохранено 1465 снимков из 1518
Сохранено 1470 снимков из 1518
Сохранено 1475 снимков из 1518
Сохранено 1480 снимков из 1518
Сохранено 1485 снимков из 1518
Сохранено 1490 снимков из 1518
Сохранено 1495 снимков из 1518
Сохранено 1500 снимков из 1518
Сохранено 1505 снимков из 1518
Сохранено 1510 снимков из 1518
Сохранено 1515 снимков из 1518
